# 트리의 앙상블
- 트리 여러 개로 예측
- 과대적합 감소

In [2]:
import pandas as pd
df = pd.read_csv('https://bit.ly/wine_csv_data')
df.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
wine_data = df[['alcohol', 'sugar', 'pH']].to_numpy()
wine_target = df['class'].to_numpy()

In [4]:
# 훈련 세트와 테스트 세트 분리
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    wine_data, wine_target, test_size=0.2, random_state=42
)

In [ ]:
# Random Forest 분류기를 학습, 교차 검증(cross-validation)을 통해 모델의 성능(정확도 등)을 평가
# Forest -> Tree가 여러 개라...

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
scores

{'fit_time': array([0.16475987, 0.16475987, 0.16321993, 0.15869188, 0.15869188]),
 'score_time': array([0.02560115, 0.01607013, 0.02714109, 0.02513742, 0.02513742]),
 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}

In [7]:
import numpy as np
print("train_score:", np.mean(scores['train_score']))
print("test_score:", np.mean(scores['test_score']))

train_score: 0.9973541965122431
test_score: 0.8905151032797809


In [10]:
rf.fit(train_input, train_target)  # 학습
rf.feature_importances_  # 특성의 중요도가 적절히 분배됨

array([0.23167441, 0.50039841, 0.26792718])

In [13]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)  # oob_score: 샘플로 한 번도 뽑히지 않은 샘플로 검증

rf.fit(train_input, train_target)

rf.oob_score_  # 훈련에 사용되지 않은 샘플을 가지고 검증한 점수

0.8934000384837406

# 엑스트라 트리
- 부트스트랩 샘플 X
- 전체 샘플 사용
- 노드를 나눈 기준이 랜덤

In [14]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(random_state=42, n_jobs=-1)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print("train_score:", np.mean(scores['train_score']))
print("test_score:", np.mean(scores['test_score']))

train_score: 0.9974503966084433
test_score: 0.8887848893166506


# 그레디언트 부스팅 (GradientBoostingClassifier)
- 이전 트리의 오차를 보완하는 방식 - 경사하강법 사용
- 점진적인 학습을 위해서 얕은 트리 사용 (max_depth - 기본값 3)
- **과대적합(overfitting)을 보완**하고, 보다 일반적인 경향의 모델을 만듦
- 1개의 cpu만 사용(n_jobs 설정 불가) -> 느림

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print("train_score:", np.mean(scores['train_score']))
print("test_score:", np.mean(scores['test_score']))

train_score: 0.8881086892152563
test_score: 0.8720430147331015


In [17]:
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.11949946, 0.74871836, 0.13178218])

엑스트라 트리의 R^2:
> train_score: 0.9974503966084433\
test_score: 0.8887848893166506
\
그레디언트 부스팅 사용 후: \
> train_score: 0.8881086892152563\
test_score: 0.8720430147331015\
--> overfitting 보완됨

# 히스토그램 기반 그래디언트 부스팅
- 훈련세트의 입력 특성을 256개의 구간으로 나누고, 1개 - 누락 데이터에 대한 처리 사용\
- 빈도수를 가지고

In [18]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)

scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print("train_score:", np.mean(scores['train_score']))
print("test_score:", np.mean(scores['test_score']))

train_score: 0.9321723946453317
test_score: 0.8801241948619236
